In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA, TruncatedSVD

from dataclasses import dataclass
from typing import Any, Dict, Iterable, List, Optional, Tuple, Union, Optional
import pyodbc

import math
import re
import warnings

print("hello world")

hello world


In [11]:
# read xlsx into pandas
df = pd.read_excel("../isec_data/df_sample_100k.xlsx", engine="openpyxl")
df

,Unnamed: 0,serial_no,acct_opn_dt,name,dob,age,gender,marital_status,email,city,...,iai_uccstatus_dt,iai_class_download_run_no,iai_modify_dt_run_no,iai_enable_for_dwid,iai_bnk_account_flag,iai_comm_class_status,iai_bank_type,iai_ifsc_cd,iai_bnk_account_flag_changed_dt,match_id
0,0,9007305054,2017-08-19,SHARAD SINGH,1995-02-02,31,M,Y,95ssharda@gmail.com,RAE BARELI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8504297334
1,1,9007305091,2017-08-18,HIRDESH SINGH,1944-02-10,82,F,M,singhvikekkumar@gmail.com,EAST DELHI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8504295943
2,2,9007305145,2017-10-07,NIRANJAN KUMAR,1990-02-15,36,M,N,niranjankumar.singh2@gmail.com,DELHI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8504359271
3,3,9007305200,2017-07-18,ABHISHEK SHARMA,1997-09-20,28,M,Y,abhishekkharma1997@gmail.com,FARIDABAD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8504257545
4,4,9007305236,2017-08-09,ASHOK KUMAR LAVANIA,1946-12-27,79,M,Y,alavania27@gmail.com,DELHI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8504288503
5,5,9007305538,2017-06-02,SURUCHI GUPTA,1988-02-23,38,F,Y,suruchibansal2388@gmail.com,NOIDA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8504206685
6,6,9007305651,2017-06-27,SHORBAN BANERJEE,1971-12-28,54,F,Y,shorban.banerjee@gmail.com,NOIDA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8504231000
7,7,9007305877,2017-08-01,SANJEEV GHOSH,1970-03-24,56,M,Y,sanjeev@whitehouseoverseas.com,DELHI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8504273767
8,8,9007306867,2017-10-05,NIDHI BEMBI,1994-09-14,31,F,N,bembindhi@gmail.com,LUDHIANA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8504355258
9,9,9007307094,2017-07-25,RUCHI PUNEET GUPTA,1986-06-01,40,F,Y,ruchi.skyblue@gmail.com,DHULE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8504265516


In [ ]:
for column in df.columns:
    # print histogram of column if numeric
    if pd.api.types.is_numeric_dtype(df[column]) and column not in ['serial_no'] and 'dt' not in column:
        plt.figure()
        df[column].hist(bins=50)
        plt.title(column)
        plt.show()
        break


In [12]:
df.sort_values(by="neq_total_assets", ascending=False)[:10][["name", "gender", "email", "neq_total_assets"]]

,name,gender,email,neq_total_assets
17,BONTHU BRAHMA REDDY,M,reddybb@hotmail.com,1.043789e+08
4,ASHOK KUMAR LAVANIA,M,alavania27@gmail.com,8.190015e+07
21,RUKMANI RAMANATHAN,F,rukku@gmail.com,2.712788e+07
20,U KARUNAKARA SHETTY,M,ukshetty2007@gmail.com,2.110634e+07
18,MANISHA KULKARNI,F,sunil.kulkarni@hotmail.com,1.405889e+07
22,NISHANT BARANWAL,M,nishant.baranwal@gmail.com,7.066600e+06
23,KISHORE JHAVERCHAND CH,M,kjchand@gmail.com,6.106202e+06
19,RISHI PAL,M,rishipal@hotmail.com,4.733602e+06
1,HIRDESH SINGH,F,singhvikekkumar@gmail.com,4.377192e+06
16,PARTHA NARAYAN,M,partha@amcoca.in,4.254955e+06


In [13]:
df.sort_values(by="eq_total_assets", ascending=False)[:10][["name", "gender", "email", "neq_total_assets"]]

,name,gender,email,neq_total_assets
23,KISHORE JHAVERCHAND CH,M,kjchand@gmail.com,6106202.49
1,HIRDESH SINGH,F,singhvikekkumar@gmail.com,4377192.30
0,SHARAD SINGH,M,95ssharda@gmail.com,280080.97
3,ABHISHEK SHARMA,M,abhishekkharma1997@gmail.com,2327.36
4,ASHOK KUMAR LAVANIA,M,alavania27@gmail.com,81900153.57
5,SURUCHI GUPTA,F,suruchibansal2388@gmail.com,683138.63
2,NIRANJAN KUMAR,M,niranjankumar.singh2@gmail.com,484907.69
7,SANJEEV GHOSH,M,sanjeev@whitehouseoverseas.com,724198.60
8,NIDHI BEMBI,F,bembindhi@gmail.com,139520.56
10,SITA KAUSHAL,F,kavs_kan@yahoo.com,1528703.81
